In [1]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers

Using TensorFlow backend.


In [2]:
from keras.applications.vgg19 import VGG19

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
# normalize
x_train = x_train / 255
x_test = x_test / 255

In [37]:
# one-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
# Image enlargement
import scipy.misc
big_x_train = np.array([scipy.misc.imresize(x_train[i], (64, 64, 3)) 
                            for i in range(len(x_train))]).astype('float32')

In [7]:
img_rows, img_cols, channels = big_x_train.shape[1], big_x_train.shape[2], big_x_train.shape[3]
img_rows, img_cols, channels

(64, 64, 3)

In [8]:
vgg_model = VGG19(include_top = False, input_shape = (img_rows, img_cols, channels))

In [9]:
# Generate features from VGG16 model
X_train_vgg_features = vgg_model.predict(big_x_train)

In [10]:
def extract_images_features(imgs, target_size, model):
    """Increase image size to target size and 
    extract image features from model"""
    large_imgs = np.array([scipy.misc.imresize(img, target_size) for img in imgs])
    features = model.predict(large_imgs)
    return features

In [11]:
X_test_vgg_features = extract_images_features(x_test, (64,64,3), vgg_model)

In [12]:
X_test_vgg_features.shape

(10000, 2, 2, 512)

### Transfer learning

In [17]:
# Transfer learning
model = models.Sequential()
model.add(layers.Conv2D(filters=128, kernel_size=2, input_shape=X_train_vgg_features.shape[1:]))
model.add(layers.Dropout(0.4))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))

In [18]:
model.compile(optimizer='rmsprop',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])
# Training
model.fit(X_train_vgg_features, y_train, epochs=20, batch_size=128, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 1s - loss: 7.0153 - acc: 0.5190 - val_loss: 5.3445 - val_acc: 0.6310
Epoch 2/20
40000/40000 [==============================] - 1s - loss: 5.7071 - acc: 0.6149 - val_loss: 5.1797 - val_acc: 0.6498
Epoch 3/20
40000/40000 [==============================] - 1s - loss: 5.4236 - acc: 0.6372 - val_loss: 5.0538 - val_acc: 0.6619
Epoch 4/20
40000/40000 [==============================] - 1s - loss: 5.3001 - acc: 0.6473 - val_loss: 4.9673 - val_acc: 0.6691
Epoch 5/20
40000/40000 [==============================] - 1s - loss: 5.1623 - acc: 0.6580 - val_loss: 4.9577 - val_acc: 0.6705
Epoch 6/20
40000/40000 [==============================] - 1s - loss: 5.0340 - acc: 0.6668 - val_loss: 4.8456 - val_acc: 0.6787
Epoch 7/20
40000/40000 [==============================] - 1s - loss: 5.0205 - acc: 0.6692 - val_loss: 4.8441 - val_acc: 0.6810
Epoch 8/20
40000/40000 [==============================] - 1s 

In [19]:
loss, accuracy = model.evaluate(X_test_vgg_features, y_test)
accuracy

 9280/10000 [==========================>...] - ETA: 0s

0.69740000000000002

### Inception model

In [28]:
img_rows, img_cols, channels = 139, 139, 3

from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channels))

In [29]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [30]:
big_x_train = np.array([scipy.misc.imresize(img, (139, 139, 3)) 
                            for img in x_train]).astype('float32')
inception_input_train = preprocess_input(big_x_train)

In [31]:
inception_input_train.shape

(50000, 139, 139, 3)

In [33]:
X_train_inception_features = inception_model.predict(inception_input_train)

In [40]:
# np.savez('inception_features_train', features=X_train_inception_features)

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [23]:
# X_train_inception_features = extract_images_features(x_train, (img_rows, img_cols, channels), inception_model)
# X_test_inception_features = extract_images_features(x_test, (img_rows, img_cols, channels), inception_model)

#### Model building, loss function and optimizer

In [35]:
# Transfer learning
model = models.Sequential()
model.add(layers.Conv2D(filters=128, kernel_size=2, input_shape=X_train_inception_features.shape[1:]))
model.add(layers.Dropout(0.4))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_287 (Conv2D)          (None, 2, 2, 128)         1048704   
_________________________________________________________________
dropout_9 (Dropout)          (None, 2, 2, 128)         0         
_________________________________________________________________
global_average_pooling2d_5 ( (None, 128)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 1,049,994.0
Trainable params: 1,049,994.0
Non-trainable params: 0.0
_________________________________________________________________


#### Training

In [38]:
# Training
model.fit(X_train_inception_features, y_train, epochs=20, batch_size=128, validation_split=0.2, verbose=2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
7s - loss: 4.3467 - acc: 0.6744 - val_loss: 3.1810 - val_acc: 0.7626
Epoch 2/20
6s - loss: 3.0708 - acc: 0.7732 - val_loss: 2.8976 - val_acc: 0.7870
Epoch 3/20
6s - loss: 2.8895 - acc: 0.7904 - val_loss: 2.9985 - val_acc: 0.7885
Epoch 4/20
6s - loss: 2.7626 - acc: 0.8015 - val_loss: 3.1425 - val_acc: 0.7744
Epoch 5/20
6s - loss: 2.6526 - acc: 0.8100 - val_loss: 2.6943 - val_acc: 0.8081
Epoch 6/20
6s - loss: 2.5876 - acc: 0.8146 - val_loss: 2.6771 - val_acc: 0.8077
Epoch 7/20
6s - loss: 2.4849 - acc: 0.8230 - val_loss: 2.6520 - val_acc: 0.8122
Epoch 8/20
6s - loss: 2.4587 - acc: 0.8259 - val_loss: 2.5727 - val_acc: 0.8181
Epoch 9/20
6s - loss: 2.4573 - acc: 0.8269 - val_loss: 2.9241 - val_acc: 0.7988
Epoch 10/20
6s - loss: 2.3734 - acc: 0.8328 - val_loss: 2.9967 - val_acc: 0.7926
Epoch 11/20
6s - loss: 2.3695 - acc: 0.8337 - val_loss: 2.7432 - val_acc: 0.8088
Epoch 12/20
6s - loss: 2.3314 - acc: 0.8370 - val_loss: 2.5879 - val

#### Bottle neck feature for tests

In [41]:
big_x_test = np.array([scipy.misc.imresize(img, (139, 139, 3)) 
                            for img in x_test]).astype('float32')
inception_input_test = preprocess_input(big_x_test)

In [42]:
X_test_inception_features = inception_model.predict(inception_input_test)

In [43]:
loss, accuracy = model.evaluate(X_test_inception_features, y_test)
accuracy

 9760/10000 [============================>.] - ETA: 0s

0.82450000000000001